## Speed comparison using the Piola mapping and without

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from ngsolve.krylovspace import CGSolver
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm

In [ ]:
def compute_time(mesh, order, geom_free):
    fes_p = L2(mesh, order=order+1, all_dofs_together=True) 
    fes_u = VectorL2(mesh, order=order, piola=True)
    fes_tr = FacetFESpace(mesh, order=order+1)
    traceop = fes_p.TraceOperator(fes_tr, average=True) 
    fes = fes_p * fes_u
    p1, q1 = fes_p.TnT()
    u1, v1 = fes_u.TnT()

    n = specialcf.normal(2)

    phat1, qhat1 = fes_tr.TnT()
    Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = geom_free)
    Bel += grad(p1)*v1 * dx -p1*(v1*n) * dx(element_boundary=True)
    Bel.Assemble()
    Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = geom_free)
    Btr += phat1 * (v1*n) *dx(element_boundary=True)
    Btr.Assemble();

    B = Bel.mat + Btr.mat @ traceop

    gfp = GridFunction(fes_p)
    gfu = GridFunction(fes_u)
    gfp.Set(1)
    start = time.perf_counter()
    gfu.vec.data = B * gfp.vec
    delta_time = time.perf_counter() - start
    return delta_time, fes_p.ndof

In [ ]:
ne = []
geom_free = []
normal = []

for i in range(12):
    mesh = Mesh(unit_square.GenerateMesh(maxh=1.5**(-i)))
    order = 4
    delta, dofs = compute_time(mesh, order, True)    
    geom_free.append(delta)
    delta, dofs = compute_time(mesh, order, False)    
    normal.append(delta)
    ne.append(dofs)
    print(i, mesh.ne, dofs)

In [ ]:
normal = np.array(normal)
normal = normal * 1000

geom_free = np.array(geom_free)
geom_free = geom_free * 1000

ne = np.array(ne)

In [ ]:
plt.plot(ne, normal, label='Normal')
plt.plot(ne, geom_free, label='Piola')
# plt.plot(ne, ne/200000)
plt.legend()
plt.title('')
plt.xlabel('ndofs')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.ylabel('Time in milliseconds')
plt.grid()

plt.savefig(f"../img/piola.png", dpi=300)
plt.show()